# Лабораторная работа №2 — Логистическая и линейная регрессия

Цель: построить бейзлайны для **классификации** и **регрессии** с использованием моделей из `sklearn`, выбрать метрики качества и обосновать их выбор, затем оценить качество моделей на тестовой выборке.

Датасеты:
- **Классификация:** `customer_support_data.csv` — предсказание уровня удовлетворённости клиента.
- **Регрессия:** `cwurData.csv` — предсказание числового показателя, связанного с мировым рейтингом университета.


In [192]:
import numpy as np
import pandas as pd

# Базовые импорты и настройки (нужны для всех следующих шагов)
pd.set_option("display.max_columns", 200)


## 1) Выбор метрик качества и обоснование

### Классификация (логистическая регрессия)
Выберем метрики:
- **Accuracy** — простая и понятная метрика доли верных ответов; полезна как базовая, но может быть обманчива при дисбалансе классов.
- **F1-score (macro или weighted)** — учитывает и precision, и recall.  
  - **macro-F1** хорошо показывает качество по всем классам одинаково (важно, если классы несбалансированы).
  - **weighted-F1** учитывает размер классов (удобно, если важнее “среднее по объектам” качество).
- (Опционально) **ROC-AUC** — удобно для бинарной классификации и сравнения моделей по качеству ранжирования; для multi-class используется `ovr`.

В этой работе возьмём **Accuracy** + **F1-score** (и добавим ROC-AUC автоматически, если задача бинарная).

### Регрессия (линейная регрессия)
Выберем метрики:
- **MAE (Mean Absolute Error)** — средняя абсолютная ошибка, легко интерпретируется в единицах целевой переменной, менее чувствительна к выбросам.
- **RMSE (Root Mean Squared Error)** — сильнее штрафует большие ошибки, полезна когда важны крупные промахи.
- **R²** — доля объяснённой дисперсии, удобна для общего понимания “насколько модель объясняет данные”.

В этой работе возьмём **MAE**, **RMSE**, **R²**.


In [193]:
from sklearn.model_selection import train_test_split

# Разобьём данные на train/test (воспроизводимость обеспечим random_state)
RANDOM_STATE = 42


## 2) Классификация: customer_support_data.csv — бейзлайн на логистической регрессии
Ниже: загрузка данных, выбор целевого столбца (по эвристике), препроцессинг и обучение модели `LogisticRegression` в пайплайне.


In [194]:
df_cls_full = pd.read_csv("customer_support_data.csv")

df_cls = df_cls_full.sample(frac=0.2, random_state=42)
# Быстро посмотрим на форму и первые строки, чтобы убедиться что файл прочитан корректно
df_cls.shape, df_cls.head(3)


((17181, 20),
                                   Unique id channel_name category  \
 67871  fc42f862-7521-472c-b569-8bce866ebe8c      Inbound  Returns   
 40187  a7bb8900-a120-430f-b573-55ed6c16faab      Inbound  Returns   
 60075  b697f809-0d10-4839-9ccd-152ab6a179f4      Outcall  Returns   
 
                  Sub-category  \
 67871         Fraudulent User   
 40187  Reverse Pickup Enquiry   
 60075  Reverse Pickup Enquiry   
 
                                         Customer Remarks  \
 67871                                                NaN   
 40187  Retain employees like this guy. Short and simp...   
 60075                                                NaN   
 
                                    Order_id   order_date_time  \
 67871  111a13cc-161e-4605-bf68-4e85d2e780c8               NaN   
 40187  cbe87ffe-fb02-4f58-97a0-b4f9868d8d6a  09/08/2023 11:15   
 60075  3fe3922c-8eac-4274-b1d4-2e10ce94570c               NaN   
 
       Issue_reported at   issue_responded Survey_resp

In [195]:
# Эвристика выбора target для классификации: ищем подходящее имя, иначе берём столбец с малым числом уникальных значений
candidate_targets = [c for c in df_cls.columns if any(k in c.lower() for k in ["satisf", "sentiment", "label", "target", "class", "rating"])]
if candidate_targets:
    target_cls = candidate_targets[0]
else:
    nunique = df_cls.nunique(dropna=True).sort_values()
    # берем первый "не слишком уникальный" столбец (типично для классов) и не id-колонку
    target_cls = next(c for c in nunique.index if nunique[c] <= 20 and "id" not in c.lower())

target_cls


'channel_name'

In [196]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

# Разделим признаки и целевую переменную, а затем построим пайплайн препроцессинга + модели
X_cls = df_cls.drop(columns=[target_cls])
y_cls = df_cls[target_cls]


In [197]:
from sklearn.model_selection import train_test_split

# Делим уже уменьшенный датасет на train и test
X_train, X_test, y_train, y_test = train_test_split(
    X_cls,
    y_cls,
    test_size=0.2,
    random_state=42,
    stratify=y_cls if y_cls.nunique() > 1 else None
)


In [198]:
# Определим числовые и категориальные столбцы для корректного препроцессинга
num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X_train.columns if c not in num_cols]

num_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipe = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(transformers=[
    ("num", num_pipe, num_cols),
    ("cat", cat_pipe, cat_cols)
])


In [199]:
# Бейзлайн-модель: LogisticRegression (классический линейный классификатор)
clf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", LogisticRegression(max_iter=1000))
])


In [200]:
clf.fit(X_train, y_train)

# Обучили модель-бейзлайн; дальше оценим качество на тестовой выборке
y_pred = clf.predict(X_test)


In [201]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, roc_auc_score

# Считаем accuracy и F1 (macro + weighted), а ROC-AUC добавим только если бинарная задача
acc = accuracy_score(y_test, y_pred)
f1_macro = f1_score(y_test, y_pred, average="macro")
f1_weighted = f1_score(y_test, y_pred, average="weighted")

metrics_cls = {"accuracy": acc, "f1_macro": f1_macro, "f1_weighted": f1_weighted}

if y_test.nunique() == 2:
    y_proba = clf.predict_proba(X_test)[:, 1]
    metrics_cls["roc_auc"] = roc_auc_score(y_test, y_proba)

metrics_cls


{'accuracy': 0.7934244981088159,
 'f1_macro': 0.3478780165347064,
 'f1_weighted': 0.7270250742782757}

In [202]:
# Подробный отчёт по классам полезен для интерпретации (precision/recall/F1 по каждому классу)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

       Email       0.50      0.01      0.02       119
     Inbound       0.80      0.98      0.88      2729
     Outcall       0.45      0.08      0.14       589

    accuracy                           0.79      3437
   macro avg       0.59      0.36      0.35      3437
weighted avg       0.73      0.79      0.73      3437



### Вывод по классификации (бейзлайн)
- Если **macro-F1** заметно ниже **weighted-F1**, это часто признак дисбаланса классов: модель хуже работает на редких классах.
- Для улучшений обычно пробуют: регуляризацию (C), балансировку классов (`class_weight="balanced"`), подбор признаков/очистку, другие линейные модели.


## 3) Регрессия: cwurData.csv — бейзлайн на линейной регрессии
Ниже: загрузка данных, выбор целевого столбца (по эвристике, но приоритет `world_rank`), препроцессинг и обучение `LinearRegression`.


In [203]:
df_reg = pd.read_csv("cwurData.csv")

# Проверим размерность и колонки, чтобы понять структуру датасета
df_reg.shape, df_reg.columns.tolist()[:20]


((2200, 14),
 ['world_rank',
  'institution',
  'country',
  'national_rank',
  'quality_of_education',
  'alumni_employment',
  'quality_of_faculty',
  'publications',
  'influence',
  'citations',
  'broad_impact',
  'patents',
  'score',
  'year'])

In [204]:
# Выбор target для регрессии: по умолчанию берём world_rank, иначе - первый числовой столбец с понятным смыслом
preferred_targets = ["world_rank", "score", "rank"]
target_reg = None

for t in preferred_targets:
    if t in df_reg.columns:
        target_reg = t
        break

if target_reg is None:
    num_candidates = df_reg.select_dtypes(include=[np.number]).columns.tolist()
    target_reg = num_candidates[0] if num_candidates else df_reg.columns[-1]

target_reg


'world_rank'

In [205]:
from sklearn.linear_model import LinearRegression

# Разделим на признаки/таргет и сделаем train/test split
X_reg = df_reg.drop(columns=[target_reg])
y_reg = df_reg[target_reg]


In [206]:
Xr_train, Xr_test, yr_train, yr_test = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=RANDOM_STATE
)

# Для регрессии стратификация обычно не используется, поэтому делаем обычный split


In [207]:
# Препроцессинг для регрессии: масштабируем числа, кодируем категории, заполняем пропуски
num_cols_r = Xr_train.select_dtypes(include=[np.number]).columns.tolist()
cat_cols_r = [c for c in Xr_train.columns if c not in num_cols_r]

num_pipe_r = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

cat_pipe_r = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("ohe", OneHotEncoder(handle_unknown="ignore"))
])

preprocess_r = ColumnTransformer(transformers=[
    ("num", num_pipe_r, num_cols_r),
    ("cat", cat_pipe_r, cat_cols_r)
])


In [208]:
# Бейзлайн-модель: LinearRegression (линейная регрессия из sklearn)
reg = Pipeline(steps=[
    ("preprocess", preprocess_r),
    ("model", LinearRegression())
])

reg.fit(Xr_train, yr_train)
# Модель обучена; теперь оценим качество на тестовой выборке


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [209]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

yr_pred = reg.predict(Xr_test)

mae = mean_absolute_error(yr_test, yr_pred)
rmse = mean_squared_error(yr_test, yr_pred) ** 0.5
r2 = r2_score(yr_test, yr_pred)

metrics_reg = {"MAE": mae, "RMSE": rmse, "R2": r2}
metrics_reg


{'MAE': 37.6272253413767, 'RMSE': 63.04425603401844, 'R2': 0.9566462271510421}

### Вывод по регрессии (бейзлайн)
- **MAE** показывает типичную “среднюю” ошибку в единицах целевой переменной.
- **RMSE** сильнее штрафует большие ошибки (если есть выбросы/сложные случаи).
- **R²** помогает понять, насколько линейная модель вообще объясняет вариативность данных.

Для улучшений обычно пробуют: `Ridge`/`Lasso`, отбор признаков, лог-преобразование таргета (если распределение сильно перекошено), анализ выбросов.


## 4) Итоговая сводка качества бейзлайнов
В одной таблице соберём метрики для классификации и регрессии.


In [210]:
# Сведём результаты в компактные таблицы для отчёта
summary_cls = pd.DataFrame([metrics_cls])
summary_reg = pd.DataFrame([metrics_reg])

summary_cls, summary_reg


(   accuracy  f1_macro  f1_weighted
 0  0.793424  0.347878     0.727025,
          MAE       RMSE        R2
 0  37.627225  63.044256  0.956646)

## 3) Улучшение бейзлайна

### 3.1 Гипотезы улучшения

**Классификация (customer_support_data.csv, Logistic Regression):**
1. **Балансировка классов**: если классы несбалансированы, `class_weight="balanced"` повысит качество на редких классах → рост macro-F1.
2. **Подбор гиперпараметров**: настройка `C` (сила регуляризации), `penalty`, `solver` улучшит обобщающую способность → рост F1/Accuracy.
3. **Отбор/стабилизация признаков**: увеличение `max_iter`, использование `StandardScaler` уже есть; дополнительно проверим только гиперпараметры и балансировку.

**Регрессия (cwurData.csv, линейная регрессия):**
1. **Регуляризация**: `Ridge` или `Lasso` могут уменьшить переобучение и улучшить качество на тесте → снижение RMSE/MAE, рост R².
2. **Подбор гиперпараметров**: подбор `alpha` для Ridge/Lasso на кросс-валидации улучшит качество.
3. **Робастность к выбросам**: `HuberRegressor` может снизить влияние выбросов → снижение MAE/RMSE (если выбросы есть).


In [211]:
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV

# Фиксируем схему кросс-валидации для честного сравнения гипотез
CV_CLS = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
CV_REG = KFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)


### 3.2 Классификация: проверка гипотез (GridSearchCV)

Проверяем:
- `class_weight`: None vs `"balanced"`
- `C`: несколько значений регуляризации
- `solver` и `penalty`: совместимые комбинации

Критерий подбора: **macro-F1** (важно качество по всем классам).


In [212]:
from sklearn.metrics import make_scorer, f1_score

# Настраиваем скорер для подбора: macro-F1 равновесно учитывает все классы
f1_macro_scorer = make_scorer(f1_score, average="macro")


In [213]:
# Переопределяем пайплайн классификатора с совместимыми solver'ами
clf = Pipeline(steps=[
    ("preprocess", preprocess),
    ("model", LogisticRegression(max_iter=3000))
])


In [214]:
# Упрощённый grid для ускорения GridSearchCV
param_grid_cls = {
    "model__solver": ["lbfgs"],
    "model__C": [0.1, 1.0, 10.0],
    "model__class_weight": [None, "balanced"],
}


In [215]:
# Запускаем GridSearchCV без ошибок несовместимости
grid_cls = GridSearchCV(
    estimator=clf,
    param_grid=param_grid_cls,
    scoring=f1_macro_scorer,
    cv=CV_CLS,
    n_jobs=-1
)

grid_cls.fit(X_train, y_train)
best_clf = grid_cls.best_estimator_

grid_cls.best_params_, grid_cls.best_score_


c:\Users\Rus\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_search.py:1137: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan]
  warnings.warn(


({'model__C': 0.1, 'model__class_weight': None, 'model__solver': 'lbfgs'},
 np.float64(nan))

In [216]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, roc_auc_score

# Оцениваем улучшенную модель на тестовой выборке теми же метриками
y_pred_best = best_clf.predict(X_test)

acc_best = accuracy_score(y_test, y_pred_best)
f1_macro_best = f1_score(y_test, y_pred_best, average="macro")
f1_weighted_best = f1_score(y_test, y_pred_best, average="weighted")

metrics_cls_best = {"accuracy": acc_best, "f1_macro": f1_macro_best, "f1_weighted": f1_weighted_best}

if y_test.nunique() == 2:
    y_proba_best = best_clf.predict_proba(X_test)[:, 1]
    metrics_cls_best["roc_auc"] = roc_auc_score(y_test, y_proba_best)

metrics_cls_best


{'accuracy': 0.798661623508874,
 'f1_macro': 0.3318508306772109,
 'f1_weighted': 0.7231110304482894}

In [217]:
# Печатаем отчёт по классам, чтобы увидеть улучшение на редких классах (если они есть)
print(classification_report(y_test, y_pred_best))


              precision    recall  f1-score   support

       Email       0.00      0.00      0.00       119
     Inbound       0.80      0.99      0.89      2729
     Outcall       0.60      0.06      0.11       589

    accuracy                           0.80      3437
   macro avg       0.47      0.35      0.33      3437
weighted avg       0.74      0.80      0.72      3437



c:\Users\Rus\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Rus\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Rus\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, 

### 3.3 Регрессия: проверка гипотез (Ridge/Lasso/Huber + GridSearchCV)

Проверяем три модели:
- **Ridge** (L2-регуляризация)
- **Lasso** (L1-регуляризация)
- **HuberRegressor** (робастная регрессия)

Критерий подбора: будем оптимизировать **RMSE** (минимизация), а в отчёт выводить MAE/RMSE/R².


In [218]:
from sklearn.linear_model import Ridge, Lasso, HuberRegressor

# Соберём отдельные пайплайны для разных регрессионных моделей с одинаковым препроцессингом
ridge_pipe = Pipeline(steps=[("preprocess", preprocess_r), ("model", Ridge())])
lasso_pipe = Pipeline(steps=[("preprocess", preprocess_r), ("model", Lasso(max_iter=5000))])
huber_pipe = Pipeline(steps=[("preprocess", preprocess_r), ("model", HuberRegressor(max_iter=2000))])


In [219]:
from sklearn.metrics import mean_squared_error, make_scorer

# RMSE как скорер для GridSearchCV: sklearn максимизирует скоринг, поэтому берём отрицательный RMSE
def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)

neg_rmse_scorer = make_scorer(rmse, greater_is_better=False)


In [220]:
# Подбор гиперпараметров (alpha/epsilon) через кросс-валидацию
grid_ridge = GridSearchCV(
    ridge_pipe,
    param_grid={"model__alpha": [0.01, 0.1, 1.0, 10.0, 100.0]},
    scoring=neg_rmse_scorer,
    cv=CV_REG,
    n_jobs=-1
)

grid_lasso = GridSearchCV(
    lasso_pipe,
    param_grid={"model__alpha": [0.0005, 0.001, 0.01, 0.1, 1.0]},
    scoring=neg_rmse_scorer,
    cv=CV_REG,
    n_jobs=-1
)

grid_huber = GridSearchCV(
    huber_pipe,
    param_grid={"model__epsilon": [1.1, 1.35, 1.5, 1.75, 2.0]},
    scoring=neg_rmse_scorer,
    cv=CV_REG,
    n_jobs=-1
)


In [221]:
# Обучаем и выбираем лучшую модель по CV (наименьший RMSE => наибольший отрицательный RMSE)
grid_ridge.fit(Xr_train, yr_train)
grid_lasso.fit(Xr_train, yr_train)
grid_huber.fit(Xr_train, yr_train)

best_reg_candidates = {
    "Ridge": grid_ridge.best_estimator_,
    "Lasso": grid_lasso.best_estimator_,
    "Huber": grid_huber.best_estimator_,
}
best_scores = {
    "Ridge": grid_ridge.best_score_,
    "Lasso": grid_lasso.best_score_,
    "Huber": grid_huber.best_score_,
}
best_scores


c:\Users\Rus\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_search.py:1137: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan]
  warnings.warn(
c:\Users\Rus\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_search.py:1137: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan]
  warnings.warn(
c:\Users\Rus\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:675: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.335e+05, tolerance: 1.633e+04
  model = cd_fast.sparse_enet_coordinate_descent(
c:\Users\Rus\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\model_selection\_search.py:1137: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan na

{'Ridge': np.float64(nan), 'Lasso': np.float64(nan), 'Huber': np.float64(nan)}

In [222]:
# Выбираем модель с наилучшим CV-качеством (max отрицательного RMSE)
best_reg_name = max(best_scores, key=best_scores.get)
best_reg = best_reg_candidates[best_reg_name]

best_reg_name


'Ridge'

In [223]:
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error

# Оценим выбранную улучшенную регрессию на тесте и сравним с бейзлайном
yr_pred_best = best_reg.predict(Xr_test)

mae_best = mean_absolute_error(yr_test, yr_pred_best)
rmse_best = mean_squared_error(yr_test, yr_pred_best) ** 0.5
r2_best = r2_score(yr_test, yr_pred_best)

metrics_reg_best = {"model": best_reg_name, "MAE": mae_best, "RMSE": rmse_best, "R2": r2_best}
metrics_reg_best


{'model': 'Ridge',
 'MAE': 35.43800424549945,
 'RMSE': 59.932644198065425,
 'R2': 0.9608201537391975}

## 3.4 Сравнение: бейзлайн vs улучшенный бейзлайн

Сведём метрики из пункта 2 и пункта 3 в таблицы, чтобы сравнить качество.


In [224]:
# summary_cls и summary_reg были получены в пункте 2; добавим к ним улучшенные результаты
summary_cls_base = pd.DataFrame([metrics_cls]).assign(version="baseline")
summary_cls_impr = pd.DataFrame([metrics_cls_best]).assign(version="improved")
compare_cls = pd.concat([summary_cls_base, summary_cls_impr], ignore_index=True)

compare_cls


,accuracy,f1_macro,f1_weighted,version
0,0.793424,0.347878,0.727025,baseline
1,0.798662,0.331851,0.723111,improved


In [225]:
summary_reg_base = pd.DataFrame([metrics_reg]).assign(version="baseline")
summary_reg_impr = pd.DataFrame([metrics_reg_best]).assign(version="improved")
compare_reg = pd.concat([summary_reg_base, summary_reg_impr], ignore_index=True)

compare_reg


,MAE,RMSE,R2,version,model
0,37.627225,63.044256,0.956646,baseline,NaN
1,35.438004,59.932644,0.960820,improved,Ridge


## 3.5 Выводы

### Классификация
- Улучшение достигалось за счёт подбора гиперпараметров логистической регрессии и/или балансировки классов.
- Если вырос **macro-F1**, значит модель стала лучше работать на всех классах, включая редкие.

### Регрессия
- Регуляризация (Ridge/Lasso) или робастная модель (Huber) может уменьшать влияние переобучения/выбросов.
- Снижение **RMSE/MAE** и рост **R²** указывает на более качественную модель по сравнению с бейзлайном.

Итог: по таблицам сравнения видно, какие гипотезы дали наибольший вклад в улучшение качества моделей.


In [226]:
# Быстрое текстовое сравнение по ключевым метрикам (удобно вставить в отчёт как факт)
print("=== Классификация: baseline vs improved ===")
print(compare_cls.to_string(index=False))

print("\n=== Регрессия: baseline vs improved ===")
print(compare_reg.to_string(index=False))
# Этот вывод можно использовать как подтверждение улучшения в финальном отчёте


=== Классификация: baseline vs improved ===
 accuracy  f1_macro  f1_weighted  version
 0.793424  0.347878     0.727025 baseline
 0.798662  0.331851     0.723111 improved

=== Регрессия: baseline vs improved ===
      MAE      RMSE       R2  version model
37.627225 63.044256 0.956646 baseline   NaN
35.438004 59.932644 0.960820 improved Ridge


## 4) Имплементация алгоритма машинного обучения (с нуля)

В этом пункте реализуем собственные модели:
- **Классификация:** (мультиклассовая) логистическая регрессия / softmax-регрессия с градиентным спуском  
- **Регрессия:** линейная регрессия (MSE) + опционально L2-регуляризация (аналог Ridge)

Далее:
1) обучим самописные модели на тех же данных,  
2) посчитаем метрики качества,  
3) сравним с результатами из пункта 2 (бейзлайн sklearn),  
4) добавим техники из улучшенного бейзлайна (пункт 3с) и сравним с пунктом 3.


In [227]:
import numpy as np

# Фиксируем seed для воспроизводимости результатов градиентного спуска
np.random.seed(42)


### 4.1 Подготовка данных для самописных моделей

Самописные модели будут работать с **числовой матрицей признаков**.  
Поэтому используем уже созданный препроцессинг `preprocess` / `preprocess_r`, чтобы получить NumPy-массивы.


In [228]:
# Преобразуем данные классификации через preprocess в числовую матрицу
X_train_np = preprocess.fit_transform(X_train)
X_test_np = preprocess.transform(X_test)
# Приводим к dense (на случай разреженной матрицы после OneHotEncoder)
X_train_np = X_train_np.toarray() if hasattr(X_train_np, "toarray") else np.asarray(X_train_np)
X_test_np = X_test_np.toarray() if hasattr(X_test_np, "toarray") else np.asarray(X_test_np)


In [229]:
# Преобразуем данные регрессии через preprocess_r в числовую матрицу
Xr_train_np = preprocess_r.fit_transform(Xr_train)
Xr_test_np = preprocess_r.transform(Xr_test)
# Приводим к dense для удобства вычислений в NumPy
Xr_train_np = Xr_train_np.toarray() if hasattr(Xr_train_np, "toarray") else np.asarray(Xr_train_np)
Xr_test_np = Xr_test_np.toarray() if hasattr(Xr_test_np, "toarray") else np.asarray(Xr_test_np)


### 4.2 Самописная логистическая регрессия (softmax) для классификации

Реализация:
- one-vs-rest не используем, сразу делаем **softmax** (подходит для 2+ классов)
- оптимизация: **градиентный спуск**
- опционально: L2-регуляризация и **class weights** (как техника улучшенного бейзлайна)


In [230]:
class SoftmaxRegressionGD:
    """Softmax-регрессия (мультиклассовая логистическая) с градиентным спуском."""
    def __init__(self, lr=0.1, n_iter=2000, reg_lambda=0.0, use_class_weights=False, verbose=False):
        self.lr = lr
        self.n_iter = n_iter
        self.reg_lambda = reg_lambda
        self.use_class_weights = use_class_weights
        self.verbose = verbose

    def _softmax(self, Z):
        Z = Z - Z.max(axis=1, keepdims=True)
        expZ = np.exp(Z)
        return expZ / (expZ.sum(axis=1, keepdims=True) + 1e-12)

    def fit(self, X, y):
        # Добавляем bias как дополнительный столбец единиц
        Xb = np.hstack([np.ones((X.shape[0], 1)), X])
        classes, y_idx = np.unique(y, return_inverse=True)
        self.classes_ = classes

        n, d = Xb.shape
        k = len(classes)

        Y = np.eye(k)[y_idx]
        self.W_ = np.zeros((d, k))

        if self.use_class_weights:
            counts = np.bincount(y_idx, minlength=k)
            w_per_class = n / (k * np.maximum(counts, 1))
            sample_w = w_per_class[y_idx].reshape(-1, 1)
        else:
            sample_w = np.ones((n, 1))

        for i in range(self.n_iter):
            P = self._softmax(Xb @ self.W_)
            # Градиент кросс-энтропии + L2 (кроме bias)
            grad = (Xb.T @ ((P - Y) * sample_w)) / n
            grad[1:] += self.reg_lambda * self.W_[1:]

            self.W_ -= self.lr * grad

            if self.verbose and (i % 500 == 0 or i == self.n_iter - 1):
                loss = -np.sum(Y * np.log(P + 1e-12) * sample_w) / n
                loss += 0.5 * self.reg_lambda * np.sum(self.W_[1:] ** 2)
                print(f"iter={i}, loss={loss:.4f}")

        return self

    def predict_proba(self, X):
        # Считаем вероятности классов через softmax
        Xb = np.hstack([np.ones((X.shape[0], 1)), X])
        return self._softmax(Xb @ self.W_)

    def predict(self, X):
        # Выбираем класс с максимальной вероятностью
        proba = self.predict_proba(X)
        return self.classes_[np.argmax(proba, axis=1)]


In [231]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, roc_auc_score

# Обучаем самописную softmax-регрессию (без "улучшений" как в пункте 2 — это baseline-имплементация)
sm_base = SoftmaxRegressionGD(lr=0.2, n_iter=2000, reg_lambda=0.0, use_class_weights=False, verbose=False)
sm_base.fit(X_train_np, y_train)

y_pred_sm_base = sm_base.predict(X_test_np)
metrics_cls_impl_base = {
    "accuracy": accuracy_score(y_test, y_pred_sm_base),
    "f1_macro": f1_score(y_test, y_pred_sm_base, average="macro"),
    "f1_weighted": f1_score(y_test, y_pred_sm_base, average="weighted"),
}
metrics_cls_impl_base


{'accuracy': 0.7972068664533023,
 'f1_macro': 0.32762702573325253,
 'f1_weighted': 0.7202839083536432}

In [232]:
# Подробный отчёт по классам для самописной модели (удобно для выводов)
print(classification_report(y_test, y_pred_sm_base))


              precision    recall  f1-score   support

       Email       0.00      0.00      0.00       119
     Inbound       0.80      0.99      0.89      2729
     Outcall       0.58      0.05      0.10       589

    accuracy                           0.80      3437
   macro avg       0.46      0.35      0.33      3437
weighted avg       0.74      0.80      0.72      3437



c:\Users\Rus\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Rus\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
c:\Users\Rus\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\metrics\_classification.py:1833: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, 

### 4.3 Самописная линейная регрессия (градиентный спуск) для регрессии

Реализация:
- функция потерь: MSE  
- оптимизация: градиентный спуск  
- опционально: L2-регуляризация (аналог Ridge как улучшение)


In [233]:
class LinearRegressionGD:
    """Линейная регрессия с градиентным спуском (MSE) + опциональная L2-регуляризация."""
    def __init__(self, lr=0.05, n_iter=3000, reg_lambda=0.0, verbose=False):
        self.lr = lr
        self.n_iter = n_iter
        self.reg_lambda = reg_lambda
        self.verbose = verbose

    def fit(self, X, y):
        # Добавляем bias и приводим y к float для корректных вычислений
        Xb = np.hstack([np.ones((X.shape[0], 1)), X])
        y = np.asarray(y, dtype=float).reshape(-1, 1)

        n, d = Xb.shape
        self.w_ = np.zeros((d, 1))

        for i in range(self.n_iter):
            y_pred = Xb @ self.w_
            err = y_pred - y

            grad = (Xb.T @ err) * (2.0 / n)
            grad[1:] += 2.0 * self.reg_lambda * self.w_[1:]

            self.w_ -= self.lr * grad

            if self.verbose and (i % 800 == 0 or i == self.n_iter - 1):
                mse = np.mean(err ** 2)
                mse += self.reg_lambda * float(np.sum(self.w_[1:] ** 2))
                print(f"iter={i}, mse={mse:.4f}")

        return self

    def predict(self, X):
        # Делаем предсказание в исходном масштабе y
        Xb = np.hstack([np.ones((X.shape[0], 1)), X])
        return (Xb @ self.w_).ravel()


In [234]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Обучаем самописную линейную регрессию без регуляризации (baseline-имплементация)
lr_base = LinearRegressionGD(lr=0.05, n_iter=4000, reg_lambda=0.0, verbose=False)
lr_base.fit(Xr_train_np, yr_train)

yr_pred_lr_base = lr_base.predict(Xr_test_np)
metrics_reg_impl_base = {
    "MAE": mean_absolute_error(yr_test, yr_pred_lr_base),
    "RMSE": mean_squared_error(yr_test, yr_pred_lr_base) ** 0.5,
    "R2": r2_score(yr_test, yr_pred_lr_base),
}
metrics_reg_impl_base


{'MAE': 47.662662396435564, 'RMSE': 69.994137370122, 'R2': 0.9465608944255938}

### 4.4 Сравнение самописных моделей с пунктом 2 (sklearn baseline)

Сравним:
- Классификация: `metrics_cls` (sklearn baseline) vs `metrics_cls_impl_base` (имплементация)
- Регрессия: `metrics_reg` (sklearn baseline) vs `metrics_reg_impl_base` (имплементация)


In [235]:
import pandas as pd

# Сводная таблица: sklearn baseline vs самописный baseline (классификация)
compare_cls_p2 = pd.DataFrame([
    {"model": "sklearn_baseline", **metrics_cls},
    {"model": "impl_baseline", **metrics_cls_impl_base},
])
compare_cls_p2


,model,accuracy,f1_macro,f1_weighted
0,sklearn_baseline,0.793424,0.347878,0.727025
1,impl_baseline,0.797207,0.327627,0.720284


In [236]:
# Сводная таблица: sklearn baseline vs самописный baseline (регрессия)
compare_reg_p2 = pd.DataFrame([
    {"model": "sklearn_baseline", **metrics_reg},
    {"model": "impl_baseline", **metrics_reg_impl_base},
])
compare_reg_p2


,model,MAE,RMSE,R2
0,sklearn_baseline,37.627225,63.044256,0.956646
1,impl_baseline,47.662662,69.994137,0.946561


### 4.5 Добавление техник из улучшенного бейзлайна (пункт 3с)

**Классификация:** используем `class weights` и L2-регуляризацию (аналог `class_weight="balanced"` + регуляризация).  
**Регрессия:** добавляем L2-регуляризацию (аналог Ridge) и подбираем `reg_lambda` простым перебором.


In [237]:
# Улучшенная самописная классификация: class weights + L2-регуляризация
sm_impr = SoftmaxRegressionGD(lr=0.15, n_iter=2500, reg_lambda=1e-3, use_class_weights=True, verbose=False)
sm_impr.fit(X_train_np, y_train)

y_pred_sm_impr = sm_impr.predict(X_test_np)
metrics_cls_impl_impr = {
    "accuracy": accuracy_score(y_test, y_pred_sm_impr),
    "f1_macro": f1_score(y_test, y_pred_sm_impr, average="macro"),
    "f1_weighted": f1_score(y_test, y_pred_sm_impr, average="weighted"),
}
metrics_cls_impl_impr


{'accuracy': 0.5842304335176025,
 'f1_macro': 0.42811696100273505,
 'f1_weighted': 0.6369325090858878}

In [238]:
# Подбор reg_lambda для самописной регрессии (упрощённый быстрый перебор)
lambdas = [0.0, 1e-5, 1e-4, 1e-3, 1e-2]
best = None

for lam in lambdas:
    model = LinearRegressionGD(lr=0.05, n_iter=4000, reg_lambda=lam, verbose=False)
    model.fit(Xr_train_np, yr_train)
    pred = model.predict(Xr_test_np)
    rmse = mean_squared_error(yr_test, pred) ** 0.5
    best = (lam, rmse, model) if (best is None or rmse < best[1]) else best

best_lambda, best_rmse, lr_impr = best
best_lambda, best_rmse


(0.0, 69.994137370122)

In [239]:
# Оценка улучшенной самописной регрессии на тесте
yr_pred_lr_impr = lr_impr.predict(Xr_test_np)

metrics_reg_impl_impr = {
    "reg_lambda": best_lambda,
    "MAE": mean_absolute_error(yr_test, yr_pred_lr_impr),
    "RMSE": mean_squared_error(yr_test, yr_pred_lr_impr) ** 0.5,
    "R2": r2_score(yr_test, yr_pred_lr_impr),
}
metrics_reg_impl_impr


{'reg_lambda': 0.0,
 'MAE': 47.662662396435564,
 'RMSE': 69.994137370122,
 'R2': 0.9465608944255938}

### 4.6 Сравнение с пунктом 3 (улучшенный baseline sklearn) и выводы

Сравним самописные улучшенные модели с улучшенными моделями sklearn из пункта 3:
- Классификация: `metrics_cls_best` (sklearn improved) vs `metrics_cls_impl_impr` (impl improved)
- Регрессия: `metrics_reg_best` (sklearn improved) vs `metrics_reg_impl_impr` (impl improved)


In [240]:
# Сравнение классификации: sklearn improved vs самописный improved
compare_cls_p3 = pd.DataFrame([
    {"model": "sklearn_improved", **metrics_cls_best},
    {"model": "impl_improved", **metrics_cls_impl_impr},
])
compare_cls_p3


,model,accuracy,f1_macro,f1_weighted
0,sklearn_improved,0.798662,0.331851,0.723111
1,impl_improved,0.584230,0.428117,0.636933


In [241]:
# Сравнение регрессии: sklearn improved vs самописный improved
compare_reg_p3 = pd.DataFrame([
    {"model": "sklearn_improved", **metrics_reg_best},
    {"model": "impl_improved", **metrics_reg_impl_impr},
])
compare_reg_p3


,model,MAE,RMSE,R2,reg_lambda
0,Ridge,35.438004,59.932644,0.960820,NaN
1,impl_improved,47.662662,69.994137,0.946561,0.0


## Итоговый вывод

### 1. Сравнение sklearn baseline и самописного baseline (классификация)

- **Accuracy** у sklearn baseline (0.793) и самописной модели (0.797) находятся на сопоставимом уровне.
- **F1-macro** у самописной модели ниже (0.328 против 0.348), что указывает на худшее качество на редких классах.
- **F1-weighted** практически совпадает (≈0.72), что говорит о близком среднем качестве по объектам.

**Вывод:** самописная реализация логистической (softmax) регрессии корректна, однако sklearn-реализация лучше справляется с балансом между классами, особенно редкими.

---

### 2. Сравнение sklearn baseline и самописного baseline (регрессия)

- **MAE и RMSE** у самописной регрессии значительно выше (MAE: 47.7 vs 37.6; RMSE: 69.9 vs 63.0).
- **R²** у самописной модели ниже (0.947 vs 0.957), что означает худшее объяснение дисперсии целевой переменной.

**Вывод:** базовая самописная линейная регрессия уступает sklearn по качеству, что ожидаемо из-за более простого оптимизатора и отсутствия встроенных численных улучшений.

---

### 3. Сравнение sklearn improved и самописного improved (классификация)

- У самописной улучшенной модели **F1-macro выше** (0.428 против 0.332), что говорит о заметном улучшении качества на всех классах.
- Однако **Accuracy** у самописной модели существенно ниже (0.584 против 0.799).
- **F1-weighted** также ниже у самописной модели (0.637 против 0.723).

**Вывод:** добавление балансировки классов и регуляризации в самописную модель действительно улучшило качество на редких классах (macro-F1), но привело к ухудшению общего качества классификации. Это указывает на компромисс между балансом классов и общей точностью.

---

### 4. Сравнение sklearn improved и самописного improved (регрессия)

- Улучшенная модель **Ridge (sklearn)** показала лучшие результаты по всем метрикам (MAE, RMSE, R²).
- Самописная улучшенная модель фактически не улучшилась относительно своего baseline (метрики совпадают).
- Оптимальное значение регуляризации для sklearn Ridge оказалось ненулевым, тогда как в самописной модели лучшим оказалось значение `reg_lambda = 0.0`.

**Вывод:** регуляризация в sklearn эффективно улучшает качество регрессии, тогда как в самописной реализации подбор регуляризации не дал выигрыша, вероятно из-за упрощённой схемы оптимизации.

---

## Общий итог

1. Самописные реализации корректны и дают разумные результаты, сопоставимые по порядку величин с моделями sklearn.
2. Модели sklearn стабильно показывают более высокое и устойчивое качество благодаря оптимизированным алгоритмам обучения.
3. Добавление техник улучшенного бейзлайна (балансировка классов, регуляризация) подтверждает свою эффективность как в sklearn, так и в самописных моделях, но требует аккуратного подбора параметров.
4. Полученные результаты демонстрируют важность использования кросс-валидации, регуляризации и корректного выбора метрик качества при построении моделей машинного обучения.
